# Jujutsu Kaisen hands detector

## Connect google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
NPZ_PATH = "/content/drive/MyDrive/Portfolio/jjk_hands_detector/model/files/dataset_npz_complete.npz"
NPZ_PATH2 = "/content/drive/MyDrive/Portfolio/jjk_hands_detector/model/files/dataset_npz_complete2.npz"

## Preprocess Data and Create labels and Features

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import os
import numpy as np

In [ ]:
data = np.load(NPZ_PATH)
X1 = data['X']
labels1 = data['y']

In [ ]:
data2 = np.load(NPZ_PATH2)
X2 = data2['X']
labels2 = data2['y']

In [ ]:
(X1.shape,labels1.shape),(X2.shape,labels2.shape)

(((180, 30, 258), (180,)), ((180, 30, 258), (180,)))

In [ ]:
X = np.concatenate((X1,X2))
labels = np.concatenate((labels1,labels2))
X.shape, labels.shape

((360, 30, 258), (360,))

In [ ]:
y = to_categorical(labels).astype(int)
y

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]])

In [ ]:
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4,

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.05,random_state=42)

In [ ]:
(X_train.shape, y_train.shape), (X_test.shape,y_test.shape)

(((342, 30, 258), (342, 6)), ((18, 30, 258), (18, 6)))

## Create a validation set

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.1,random_state=42)

In [ ]:
(X_train.shape,y_train.shape),(X_val.shape,y_val.shape)

(((307, 30, 258), (307, 6)), ((35, 30, 258), (35, 6)))

## Build and Train LSTM Neural Network

### Experiment 1:
* 1 LSTM layer
* Early stopping
* Optimizer Adama 0.001 (default)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
early_stopping = EarlyStopping(monitor="val_accuracy",
                               patience=15,
                               mode="max",
                               restore_best_weights=True)

In [ ]:
model = Sequential(
                   [LSTM(64,return_sequences=False,activation="relu",input_shape=(30,258)),
                    Dense(64,activation="relu"),
                    Dense(32,activation="relu"),
                    Dense(6,activation="softmax")]
                  )


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
model.fit(X_train,
          y_train,
          validation_data=(X_val,y_val),
          epochs=300,
          callbacks=[early_stopping])

Epoch 1/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 331ms/step - accuracy: 0.2308 - loss: 1.8243 - val_accuracy: 0.2000 - val_loss: 1.6667
Epoch 2/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4236 - loss: 1.3061 - val_accuracy: 0.3714 - val_loss: 1.3434
Epoch 3/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5945 - loss: 1.0350 - val_accuracy: 0.5429 - val_loss: 1.2355
Epoch 4/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6777 - loss: 0.7168 - val_accuracy: 0.6286 - val_loss: 1.1368
Epoch 5/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7862 - loss: 0.5879 - val_accuracy: 0.8000 - val_loss: 0.7968
Epoch 6/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7221 - loss: 0.6173 - val_accuracy: 0.8286 - val_loss: 0.5949
Epoch 7/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8520 - loss: 0.5093 - val_accuracy: 0.7429 - val_loss: 0.7038
Epoch 8/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7359 - loss: 0.6392 - val_accuracy: 0

### Experiment 2

* 2 LSTM lkayers
* Reduce Learning Rate

In [ ]:
lr_cb = ReduceLROnPlateau(monitor="val_accuracy",
                          factor=0.5,
                          patience=3,
                          min_lr=1e-6)

In [ ]:
model2 = Sequential([
    LSTM(64,return_sequences=True,activation="relu",input_shape=(30,258)),
    LSTM(64,return_sequences=False,activation="relu"),
    Dense(64,activation="relu"),
    Dense(32,activation="relu"),
    Dense(6,activation="softmax")

])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model2.compile(optimizer=tf.keras.optimizers.Adam(0.001),
               loss="categorical_crossentropy",
               metrics=["accuracy"])

In [ ]:
model2.fit(X_train,y_train,
           validation_data=(X_val,y_val),
           epochs=50,
           callbacks=[lr_cb])

Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 454ms/step - accuracy: 0.1590 - loss: 1.8324 - val_accuracy: 0.1429 - val_loss: 1.9792 - learning_rate: 0.0010
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.3842 - loss: 1.5221 - val_accuracy: 0.3143 - val_loss: 1.6418 - learning_rate: 0.0010
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5226 - loss: 1.2241 - val_accuracy: 0.4286 - val_loss: 1.1892 - learning_rate: 0.0010
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6180 - loss: 0.8799 - val_accuracy: 0.7143 - val_loss: 0.7733 - learning_rate: 0.0010
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6897 - loss: 0.9115 - val_accuracy: 0.7714 - val_loss: 0.8607 - learning_rate: 0.0010
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7429 - loss: 0.6163 - val_accuracy: 0.8571 - val_loss: 0.7329 - learning_rate: 0.0010
Epoch 7/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9049 - loss: 0.3609 - val_ac

### Experiment 3

* Early stopping (val_loss)
* 2 LSTM layers

In [ ]:
cb_early_stopping = EarlyStopping(monitor="val_loss",
                               patience=15,
                               mode="min",
                               restore_best_weights=True)

In [ ]:
model3 = Sequential([
    LSTM(64,return_sequences=True,activation="relu",input_shape=(30,258)),
    LSTM(64,return_sequences=False,activation="relu"),
    Dense(64,activation="relu"),
    Dense(32,activation="relu"),
    Dense(6,activation="softmax")
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model3.compile(optimizer="Adam",
               loss="categorical_crossentropy",
               metrics=["accuracy"])

In [ ]:
model3.fit(X_train,
           y_train,
           validation_data=(X_val,y_val),
           epochs=50,
           callbacks=[cb_early_stopping])

Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 370ms/step - accuracy: 0.1930 - loss: 1.7390 - val_accuracy: 0.2000 - val_loss: 1.8561
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.4321 - loss: 1.3535 - val_accuracy: 0.3429 - val_loss: 1.3187
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5149 - loss: 1.0149 - val_accuracy: 0.5143 - val_loss: 0.9951
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6056 - loss: 0.8339 - val_accuracy: 0.6571 - val_loss: 0.9585
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6333 - loss: 0.7834 - val_accuracy: 0.4000 - val_loss: 1.0703
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6163 - loss: 0.9317 - val_accuracy: 0.4000 - val_loss: 1.3602
Epoch 7/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5262 - loss: 1.1302 - val_accuracy: 0.6000 - val_loss: 1.0176
Epoch 8/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6010 - loss: 0.9191 - val_accuracy: 0.5714 - 

### Experiment 4:

* 1 LSTM layer
* Early stopping monitoring val_accuracy
* Reduce learning rate

In [ ]:
lr_cb = ReduceLROnPlateau(monitor="val_accuracy",
                          factor=0.5,
                          patience=3,
                          min_lr=1e-6)

In [ ]:
model4 = Sequential([
    tf.keras.layers.InputLayer(shape=(30,258)),
    LSTM(64,return_sequences=False,activation="relu"),
    Dense(64,activation="relu"),
    Dense(32,activation="relu"),
    Dense(6,activation="softmax")
])

In [ ]:
model4.compile(optimizer=tf.keras.optimizers.Adam(0.001),
               loss="categorical_crossentropy",
               metrics=["accuracy"])

In [ ]:
model4.fit(X_train,
           y_train,
           validation_data=(X_val,y_val),
           epochs=50,
           callbacks=[early_stopping,lr_cb])

Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 281ms/step - accuracy: 0.1982 - loss: 2.0355 - val_accuracy: 0.2000 - val_loss: 1.7533 - learning_rate: 0.0010
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.3227 - loss: 1.5772 - val_accuracy: 0.3714 - val_loss: 1.4674 - learning_rate: 0.0010
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4288 - loss: 1.3898 - val_accuracy: 0.5429 - val_loss: 1.3383 - learning_rate: 0.0010
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6507 - loss: 1.0788 - val_accuracy: 0.5714 - val_loss: 1.0510 - learning_rate: 0.0010
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6744 - loss: 0.9007 - val_accuracy: 0.5714 - val_loss: 0.8565 - learning_rate: 0.0010
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6719 - loss: 0.8343 - val_accuracy: 0.3714 - val_loss: 1.3617 - learning_rate: 0.0010
Epoch 7/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5921 - loss: 0.9392 - val_ac

## Saving the Weights for the model 1 (Best model)

In [ ]:
model.save_weights("/content/drive/MyDrive/Portfolio/jjk_hands_detector/model/files/best_weigths.weights.h5")

In [ ]:
model.evaluate(X_test,y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 633ms/step - accuracy: 0.9444 - loss: 0.0633


[0.06325709074735641, 0.9444444179534912]

In [ ]:
new_model = tf.keras.models.clone_model(model)

In [ ]:
new_model.compile(optimizer="Adam",
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

In [ ]:
new_model.evaluate(X_test,y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 822ms/step - accuracy: 0.9444 - loss: 0.0633


[0.06325709074735641, 0.9444444179534912]

In [ ]:
new_model.load_weights("/content/drive/MyDrive/Portfolio/jjk_hands_detector/model/files/best_weigths.weights.h5")

In [ ]:
new_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        82,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 89,126 (348.15 KB)

 Trainable params: 89,126 (348.15 KB)

 Non-trainable params: 0 (0.00 B)

## Make Predictions

In [ ]:
characters = ["Gojo","Hakari","Megumi","Sukuna","Yuji","Yuta"]

In [ ]:
res = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


In [ ]:
characters[np.argmax(res[8])]

'Megumi'

In [ ]:
characters[np.argmax(y_test[8])]

'Megumi'

## Evaluate using Confusion Matrix and Accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
y_hat = new_model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


In [ ]:
y_true = np.argmax(y_test,axis=1).tolist()
y_hat = np.argmax(y_hat,axis=1).tolist()

In [ ]:
y_true

[1, 1, 3, 4, 1, 3, 0, 1, 2, 4, 2, 5, 1, 5, 3, 4, 0, 4]

In [ ]:
y_hat

[1, 1, 3, 4, 1, 3, 0, 1, 2, 4, 2, 5, 1, 5, 3, 4, 0, 3]

In [ ]:
multilabel_confusion_matrix(y_true,y_hat)

array([[[16,  0],
        [ 0,  2]],

       [[13,  0],
        [ 0,  5]],

       [[16,  0],
        [ 0,  2]],

       [[14,  1],
        [ 0,  3]],

       [[14,  0],
        [ 1,  3]],

       [[16,  0],
        [ 0,  2]]])

In [ ]:
accuracy_score(y_true,y_hat)

0.9444444444444444